In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp
import time
import math

# --- 1. Inicialización de Componentes ---

# Inicializar MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False,
                    model_complexity=0,  # 0: Lite, 1: Full, 2: Heavy
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# --- 2. Simulación de Carga de Modelo TFLite (OE 2) ---

def load_tflite_model(model_path):
    """
    Carga un modelo TFLite y asigna los tensores.
    Simula la carga del modelo facial optimizado (INT8).
    """
    try:
        # Cargar el intérprete TFLite
        interpreter = tf.lite.Interpreter(model_path=model_path)
        interpreter.allocate_tensors()

        # Obtener detalles de entrada y salida
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()

        print(f"Modelo TFLite cargado: {model_path}")
        print(f"Detalles de Entrada: {input_details[0]['shape']}, Tipo: {input_details[0]['dtype']}")
        print(f"Detalles de Salida: {output_details[0]['shape']}, Tipo: {output_details[0]['dtype']}")

        return interpreter, input_details, output_details

    except ValueError:
        print(f"Error: No se pudo cargar el modelo TFLite de {model_path}.")
        print("Asegúrese de que el archivo .tflite exista.")
        print("Generando un intérprete simulado para continuar...")
        # Fallback: Crear un intérprete simulado si el archivo no existe
        return None, None, None
    except Exception as e:
        print(f"Error inesperado al cargar {model_path}: {e}")
        return None, None, None

# --- 3. Clases de Procesamiento Multimodal (OE 1 y 3) ---

class MultimodalAttentionPipeline:

    def __init__(self, facial_model_path):
        # Cargar el modelo facial (emociones)
        # Usaremos un placeholder para las clases de emoción
        self.emotion_labels = ['Confusión', 'Fatiga/Aburrimiento', 'Interés/Atención']

        # (Simulación) Cargar el modelo TFLite de emoción (ej. Mini-Xception INT8)
        # self.facial_interpreter, self.facial_input, self.facial_output = load_tflite_model(facial_model_path)

        # Placeholder si el modelo TFLite no se carga
        print("ADVERTENCIA: Modelo facial TFLite no cargado. Usando predicciones simuladas.")
        self.facial_interpreter = None

        # Cargar el modelo postural (MediaPipe)
        self.pose_estimator = mp_pose.Pose(model_complexity=0, min_detection_confidence=0.5)

    def _calculate_angle(self, a, b, c):
        """Calcula el ángulo entre tres puntos (landmarks)."""
        a = np.array(a) # Primer punto
        b = np.array(b) # Punto medio (vértice)
        c = np.array(c) # Tercer punto

        radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
        angle = np.abs(radians * 180.0 / np.pi)

        if angle > 180.0:
            angle = 360 - angle
        return angle

    def process_facial_emotion(self, face_roi):
        """
        Procesa el ROI facial para la detección de emociones.
        (Simulación de inferencia TFLite)
        """
        if self.facial_interpreter is None:
            # Simulación si el modelo no está cargado
            # Devuelve una distribución de probabilidad aleatoria
            simulated_preds = np.random.rand(len(self.emotion_labels))
            simulated_preds /= np.sum(simulated_preds)
            return simulated_preds

        # Proceso real de TFLite (requiere el modelo)
        # 1. Pre-procesar ROI (ej. 64x64, escala de grises, normalizar)
        # input_data = cv2.resize(face_roi, (64, 64))
        # input_data = cv2.cvtColor(input_data, cv2.COLOR_BGR2GRAY)
        # input_data = np.expand_dims(input_data, axis=-1)
        # input_data = np.expand_dims(input_data, axis=0)
        # input_data = input_data.astype(np.float32) / 255.0

        # 2. Establecer tensor de entrada
        # self.facial_interpreter.set_tensor(self.facial_input[0]['index'], input_data)

        # 3. Invocar inferencia
        # self.facial_interpreter.invoke()

        # 4. Obtener salida
        # preds = self.facial_interpreter.get_tensor(self.facial_output[0]['index'])[0]
        # return preds

        pass # Reemplazado por la simulación de arriba

    def process_pose_estimation(self, frame_rgb):
        """
        Procesa el frame completo para la estimación de pose con MediaPipe.
        """
        results = self.pose_estimator.process(frame_rgb)
        features = {'head_angle': 0.0, 'is_slouching': False}

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            # Obtener landmarks clave (nariz, hombros)
            try:
                nose = [landmarks[mp_pose.PoseLandmark.NOSE.value].x, landmarks[mp_pose.PoseLandmark.NOSE.value].y]
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

                # Calcular punto medio de hombros
                shoulder_mid = [(left_shoulder[0] + right_shoulder[0]) / 2, (left_shoulder[1] + right_shoulder[1]) / 2]

                # Calcular ángulo de inclinación de la cabeza (vertical)
                # (Usamos un punto artificial 'arriba' de la nariz para medir la verticalidad)
                nose_up = [nose[0], nose[1] - 0.1]

                # Ángulo: (nose_up) - (nose) - (shoulder_mid)
                angle = self._calculate_angle(nose_up, nose, shoulder_mid)
                features['head_angle'] = angle

                # Detectar encorvamiento (slouching)
                if nose[1] > shoulder_mid[1]: # Si la nariz está por debajo de los hombros
                    features['is_slouching'] = True

            except Exception as e:
                pass # Error al obtener landmarks

        return features, results.pose_landmarks

    def fuse_and_classify(self, facial_preds, pose_features):
        """
        Motor de Fusión Multimodal (Lógica de OE 3).
        Combina las salidas para determinar el estado afectivo.
        """

        # 1. Interpretar predicciones faciales
        emotion_idx = np.argmax(facial_preds)
        emotion_label = self.emotion_labels[emotion_idx]
        emotion_conf = facial_preds[emotion_idx]

        # 2. Aplicar reglas de fusión

        # Regla 1: Postura domina para Fatiga
        # (Ángulo > 30 grados indica cabeza muy inclinada)
        if pose_features['is_slouching'] or pose_features['head_angle'] > 30:
            return "Fatiga", (0, 0, 255) # Rojo

        # Regla 2: Emoción facial domina para Aburrimiento/Confusión
        if emotion_label == 'Fatiga/Aburrimiento' and emotion_conf > 0.6:
            return "Distracción (Aburrido)", (0, 165, 255) # Naranja

        if emotion_label == 'Confusión' and emotion_conf > 0.5:
            return "Confusión (Posible Duda)", (0, 255, 255) # Amarillo

        # Regla 3: Default es Atención
        return "Atención", (0, 255, 0) # Verde

# --- 4. Bucle Principal de Ejecución (Demo del Prototipo) ---

if __name__ == "__main__":

    # (Simulación OE 2) Ruta al modelo facial optimizado
    # Descargar un modelo de emoción TFLite (ej. de Kaggle o TF Hub)
    # y renombrarlo a 'emotion_model.tflite'
    FACIAL_MODEL_PATH = "emotion_model.tflite"

    pipeline = MultimodalAttentionPipeline(FACIAL_MODEL_PATH)

    # Iniciar captura de video (Cámara 0 o un archivo de video)
    # cap = cv2.VideoCapture(0)
    cap = cv2.VideoCapture("ruta_a_un_video_de_aula_virtual.mp4") # O usar un archivo

    if not cap.isOpened():
        print("Error: No se pudo abrir la fuente de video.")
        exit()

    # (Simulación OE 1) Usamos un detector de rostros simple (Haar Cascade)
    # para encontrar el ROI, ya que el modelo de emoción espera un rostro.
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    print("Iniciando pipeline... Presione 'q' para salir.")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        start_time = time.time() # Iniciar contador para FPS

        # --- Pipeline de Inferencia ---

        # 1. Convertir a RGB (requerido por MediaPipe)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # 2. Procesamiento Postural (OE 1)
        pose_features, landmarks = pipeline.process_pose_estimation(frame_rgb)

        # 3. Procesamiento Facial (OE 1)
        # Encontrar el rostro (ROI)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)

        facial_preds = np.array([0, 0, 1]) # Default: Atención

        if len(faces) > 0:
            # Usar solo el primer rostro detectado
            (x, y, w, h) = faces[0]
            face_roi = frame_rgb[y:y+h, x:x+w]

            # Inferencia del modelo de emoción (OE 2)
            facial_preds = pipeline.process_facial_emotion(face_roi)

            # Dibujar ROI facial
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        # 4. Fusión Multimodal (OE 3)
        estado_afectivo, color = pipeline.fuse_and_classify(facial_preds, pose_features)

        # --- Cálculo de Métricas (OE 4) ---
        end_time = time.time()
        latency = (end_time - start_time) * 1000 # Latencia en ms
        fps = 1.0 / (end_time - start_time) if (end_time - start_time) > 0 else 0

        # --- Visualización (Simulación Dashboard OE 3) ---

        # Dibujar landmarks posturales
        if landmarks:
            mp_drawing.draw_landmarks(
                frame, landmarks, mp_pose.POSE_CONNECTIONS)

        # Mostrar Estado Afectivo
        cv2.putText(frame, f"Estado: {estado_afectivo}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

        # Mostrar Métricas de Rendimiento (OE 4)
        cv2.putText(frame, f"Latencia: {latency:.2f} ms", (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(frame, f"FPS: {fps:.2f}", (10, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        # (Simulación Dashboard)
        # Aquí es donde se enviarían los datos (estado_afectivo, latency)
        # a un servidor Flask/Streamlit para el dashboard del docente.

        cv2.imshow('Prototipo Edge AI - Aula Híbrida', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()